In [1]:
%load_ext Cython

In [2]:
%%cython

import numpy as np
cimport numpy as np
from libc.math cimport log, exp, ceil
cimport cython
from random import random

# create the memoized list of log(n!) up to some bound
cdef np.ndarray[np.float64_t, ndim=1] gen_lfact(int n):
    cdef int i
    cdef np.ndarray[np.float64_t, ndim=1] l = np.zeros((n+1,), dtype=np.float64)
    l[0] = 0
    for i in range(1, n+1):
        l[i] = l[i-1] + log(i)
    return l

cdef np.float64_t[:] lfact

# return log(T(n, m))
cdef double ltutte(int n, int m):
    if n < 0 or m < 0:
        return -float('inf')
    global lfact
    return (n+1)*log(2) + lfact[2*m+1] + lfact[2*m+3*n] - lfact[m]*2 - lfact[n] - lfact[2*m+2*n+2]

cdef long choose(long n, long m, double logremaining_graphs):
    cdef double r = random()
    cdef double total = 0.0
    cdef long d
    cdef long c
    cdef long k
    cdef long j
    cdef long count
    cdef double amt
    if m >= (n+1):
        for d in range(n+1):
            count = ((d+1)*m)/(n+1)
            for c in range(count):
                k = d + ((-n-1)*(c+1))/m + 1
                j = c
                amt = ltutte(k, j) + ltutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                if n == n-k and m == m-j-1:
                    total -= amt
                if total >= r:
                    r = random()
                    if r < 0.5:
                        return (k << 32) + j
                    else:
                        return ((n-k) << 32) + m-j-1
    else:
        for d in range(m):
            count = ((d+1)*(n+1))/m
            for c in range(count):
                k = c
                #print d, m, c, n, ((-m)*(c+1))/(n+1)
                j = d + ((-m)*(c+1))/(n+1) + 1
                #print k, j
                amt = ltutte(k, j) + ltutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                #print total
                if n == n-k and m == m-j-1:
                    total -= amt
                if total >= r:
                    r = random()
                    if r < 0.5:
                        return (k << 32) + j
                    else:
                        return ((n-k) << 32) + m-j-1

cdef logmake_graph(internal, external):
    external_stack = [external]
    internal_stack = [internal]
    G = []
    cdef int n
    cdef int m
    cdef double logtotal_graphs
    cdef double logadd_internal
    cdef double logremaining_graphs
    while len(external_stack) > 0:
        internal = internal_stack.pop()
        external = external_stack.pop()
        n = len(internal)
        m = len(external)-2
        if n == 0 and m == 0:
            # nothing else to add
            continue
        logtotal_graphs = ltutte(n, m)
        logadd_internal = ltutte(n-1, m+1)
        if random() < exp(logadd_internal - logtotal_graphs):
            # add internal triangle
            triangle = (external[0], external[1], internal[0])
            G.append(triangle)
            external = np.insert(external, 1, internal[0])
            internal_stack.append(internal[1:])
            external_stack.append(external)
        else:
            logremaining_graphs = logtotal_graphs + log(1 - exp(logadd_internal - logtotal_graphs))
            k = choose(n, m, logremaining_graphs)
            # TODO: better canonical way to store a pair of ints?
            j = k & 0xFFFFFFFF
            k = k >> 32
            triangle = (external[0], external[1], external[j+2])
            G.append(triangle)
            left = external[1:j+3]
            right = np.append(external[j+2:], external[0])
            internal_stack.append(internal[:k])
            external_stack.append(left)
            internal_stack.append(internal[k:])
            external_stack.append(right)
    return G

def sample_graph(int n, int m):
    global lfact
    lfact = gen_lfact(5*n+5*m)
    cdef np.ndarray[np.int64_t, ndim=1] internal = np.arange(m+2, m+n+2)
    cdef np.ndarray[np.int64_t, ndim=1] external = np.arange(0, m+2)
    return logmake_graph(internal, external)

In [5]:
import random
from __future__ import print_function

def ball_size(g, n, m):
    N = m + n + 2
    adj = [set() for _ in range(N)]
    for a, b, c in g:
        adj[a].add(b)
        adj[a].add(c)
        adj[b].add(a)
        adj[b].add(c)
        adj[c].add(a)
        adj[c].add(b)
    start = random.randrange(N)
    fringe = set()
    seen = set()
    fringe.add(start)
    distance = 0
    while len(fringe) > 0:
        _fringe = set()
        for vertex in fringe:
            seen.add(vertex)
            for neighbor in adj[vertex]:
                if neighbor not in seen and neighbor not in fringe:
                    _fringe.add(neighbor)
        print(distance, len(fringe), len(seen), 1.0*len(seen)/(distance+0.00001)**4, sep='\t')
        fringe = _fringe
        distance += 1

In [9]:
from collections import Counter

def max_degree(g, n, m):
    degrees = Counter()
    for triple in g:
        degrees.update(triple)
    # count external vertices one more time
    degrees.update(range(m+2))
    return max(degrees.values())